In [1]:
import pandas as pd
import random
import numpy as np
import os
import tensorflow.keras
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet import preprocess_input as process_mobilenet
from tensorflow.keras.applications.inception_v3 import preprocess_input as process_inception
from tensorflow.keras.applications.resnet50 import preprocess_input as process_resnet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as process_vgg
import itertools

import cv2

from sklearn.metrics import confusion_matrix, accuracy_score
# Start CoLab
try:
  from google.colab import drive
  %tensorflow_version 2.x
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google CoLab")
except:
  print("Note: not using Google CoLab")
  COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [3]:
def wrap_generator(generator):
    
    while True:
        x,y = next(generator)
        y = tf.keras.utils.to_categorical(y)
        zeros = tf.zeros_like(y) + tf.constant([1.,0.])
        y = tf.concat([y,zeros], axis=0)
        yield x,y

In [4]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)



In [5]:
def get_model(train=True):
    set_seed(33)
    inp = Input((224,224,3))

    pre_1 = Lambda(process_vgg)

    mobilenet = VGG16(weights = 'imagenet', include_top = False)

    for layer in mobilenet.layers[:-4]:
        layer.trainable = False

    mobilenet_process = pre_1(GaussianNoise(0.1)(inp))
    mobile_net = mobilenet(mobilenet_process)

    noise = Lambda(tf.zeros_like)(mobile_net)
    noise = GaussianNoise(0.1)(noise)

    if train:
        x = Lambda(lambda z: tf.concat(z, axis=0))([mobile_net,noise])
        x = Activation('relu')(x)
    else:
        x = mobile_net

    x_mobilenet = GlobalMaxPool2D()(x)
    x_mobilenet = Dense(1024, activation='relu')(x_mobilenet)

    a = x_mobilenet
        
    a = Dense(512, activation='relu')(a)
    a = Dense(128, activation='relu')(a)
    out = Dense(2, activation='softmax')(a)
 
    model = Model(inp, out)
    model.compile(Adam(lr=1e-2), loss='binary_crossentropy')
    
    return model

In [6]:

### CREATE EMPTY GENERATORS ###

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

SHAPE = (224,224,3)

In [7]:
### FLOW GENERATORS ###

train_generator = train_datagen.flow_from_directory(
            '/content/drive/My Drive/Dataset_Project/Dataset 4',
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = 256,
            class_mode = 'categorical',
            shuffle = True,
            seed = 33,
            classes = ['no']
    )

test_generator = test_datagen.flow_from_directory(
            '/content/drive/My Drive/Dataset_Project/Dataset 2/',
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = 256,
            class_mode = 'categorical',
            shuffle = False,
            seed = 33,
            classes = ['no','yes']
)

Found 1500 images belonging to 1 classes.
Found 253 images belonging to 2 classes.


In [8]:
print(train_generator.class_indices)
print(test_generator.class_indices)

{'no': 0}
{'no': 0, 'yes': 1}


In [1]:


model = get_model()
model.fit(wrap_generator(train_generator), steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=5)

NameError: ignored

In [ ]:
  ### RETRIVE TEST LABEL FROM GENERATOR ###

test_num = test_generator.samples

label_test = []
for i in range((test_num // test_generator.batch_size)+1):
    X,y = test_generator.next()
    label_test.append(y)
        
label_test = np.argmax(np.vstack(label_test), axis=1)
label_test.shape
print(label_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
### SWITCH TO INFERENCE MODE TO COMPUTE PREDICTIONS ###

ineference_model = get_model(train=False)

In [ ]:
### COMPUTE PREDICTIONS ON TEST DATA ###

pred_test = np.argmax(ineference_model.predict(test_generator), axis=1)
print(pred_test)

[1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 0 1
 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0
 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1
 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1
 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0]


In [ ]:

### ACCURACY ON TEST DATA ###

print('ACCURACY:', accuracy_score(label_test, pred_test))

ACCURACY: 0.4268774703557312


In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

In [ ]:
### CONFUSION MATRIX ON TEST DATA ###

cnf_matrix = confusion_matrix(label_test, pred_test)

plt.figure(figsize=(7,7))
plot_confusion_matrix(cnf_matrix, classes=['YES','NO'])
plt.show()

In [ ]:
from sklearn .metrics import roc_auc_score 
from sklearn import metrics
print(roc_auc_score(label_test,pred_test))

0.4140552995391705


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
gaussian_noise (GaussianNoise)  (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           gaussian_noise[0][0]             
__________________________________________________________________________________________________
vgg16 (Functional)              (None, None, None, 5 14714688    lambda[0][0]                     
______________________________________________________________________________________________